In [ ]:
!python3 --version

In [ ]:
import ultralytics
ultralytics.__version__

In [ ]:
import torch
torch.__version__

In [ ]:
torch.cuda.get_device_name(0)

# Detect, track and count Persons

In [ ]:
%cd yolov8_DeepSORT

In [ ]:
from ultralytics import YOLO

import time
import torch
import cv2
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np

# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

results = model("img.jpg", save=True)



class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    probs = result.probs  # Class probabilities for classification outputs
    cls = boxes.cls.tolist()  # Convert tensor to list
    xyxy = boxes.xyxy
    xywh = boxes.xywh  # box with xywh format, (N, 4)
    coord_array = xyxy.numpy()
    reshaped_array = np.reshape(numpy_array, (-1, 1))
    print(reshaped_array)
    conf = boxes.conf
    print(cls)
    for class_index in cls:
        class_name = class_names[int(class_index)]
        print("Class:", class_name)

# DeepSORT

In [ ]:
from ultralytics import YOLO

import time
import torch
import cv2
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np

from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker

deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=70)

In [ ]:
# Define the video path
video_path = 'vid1.mp4'

cap = cv2.VideoCapture(video_path)

# Get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'output.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
frames = []

unique_track_ids = set()

In [ ]:
i = 0
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()

while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        
        og_frame = frame
        frame = og_frame.copy()

        model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

        results = model(frame, device="cpu", classes=3, conf=0.6 )

        class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

        for result in results:
            boxes = result.boxes  # Boxes object for bbox outputs
            probs = result.probs  # Class probabilities for classification outputs
            cls = boxes.cls.tolist()  # Convert tensor to list
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh  # box with xywh format, (N, 4)
            for class_index in cls:
                class_name = class_names[int(class_index)]
                print("Class:", class_name)

        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh
        bboxes_xywh = xywh.cpu().numpy()
        bboxes_xywh = np.array(bboxes_xywh, dtype=float)
        
        tracks = tracker.update(bboxes_xywh, conf, og_frame)
        
        for track in tracker.tracker.tracks:
            track_id = track.track_id
            hits = track.hits
            x1, y1, x2, y2 = track.to_tlbr()  # Get bounding box coordinates in (x1, y1, x2, y2) format
            w = x2 - x1  # Calculate width
            h = y2 - y1  # Calculate height

            # Set color values for red, blue, and green
            red_color = (0, 0, 255)  # (B, G, R)
            blue_color = (255, 0, 0)  # (B, G, R)
            green_color = (0, 255, 0)  # (B, G, R)

            # Determine color based on track_id
            color_id = track_id % 3
            if color_id == 0:
                color = red_color
            elif color_id == 1:
                color = blue_color
            else:
                color = green_color

            cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

            text_color = (0, 0, 0)  # Black color for text
            cv2.putText(og_frame, f"{class_name}-{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

            # Add the track_id to the set of unique track IDs
            unique_track_ids.add(track_id)

        # Update the person count based on the number of unique track IDs
        person_count = len(unique_track_ids)

        # Update FPS and place on frame
        current_time = time.perf_counter()
        elapsed = (current_time - start_time)
        counter += 1
        if elapsed > 1:
            fps = counter / elapsed
            counter = 0
            start_time = current_time

        # Draw person count on frame
        cv2.putText(og_frame, f"Bike Count: {person_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

        # Append the frame to the list
        frames.append(og_frame)

        # Write the frame to the output video file
        out.write(og_frame)

        # Show the frame
        cv2.imshow("Video",og_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
             break

cap.release()
out.release()
cv2.destroyAllWindows()


# Prediction

## Model importing - CNN_LSTM

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("../HCMC_vehicle__dataset.csv")
#print(list(data.columns))
# one-hot encode the class_name feature
class_name_encoded = pd.get_dummies(data['Class_name'])
# concatenate encoded class_name to the data
data = pd.concat([data, class_name_encoded], axis=1)

# # drop the original class_name column
data = data.drop(['Class_name'], axis=1)

##DATA CLEANING

data.isnull()

data.isna()
data.isna().any()
data.isna().sum()
data.isna().any().sum()

data.replace({False: 0, True: 1}, inplace=True)
data = data.fillna(value=0, method=None, axis=None, limit=None).infer_objects(copy=False)



features = ['Bicycle', 'Bus', 'Car', 'Motorcycle', 'Truck',
            'X_min', 'Y_min', 'X_max', 'Y_max',
            'X_min_1', 'Y_min_1', 'X_max_1', 'Y_max_1',
            'X_min_2', 'Y_min_2', 'X_max_2', 'Y_max_2',
            'X_min_3', 'Y_min_3', 'X_max_3', 'Y_max_3',
            'X_min_4', 'Y_min_4', 'X_max_4', 'Y_max_4',
            'X_min_5', 'Y_min_5', 'X_max_5', 'Y_max_5',
            'X_min_6', 'Y_min_6', 'X_max_6', 'Y_max_6',
            'X_min_7', 'Y_min_7', 'X_max_7', 'Y_max_7',
            ]

target = ['X_max_8', 'X_min_8', 'Y_max_8', 'Y_min_8']


X = data[features]
y = data[target]
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
timesteps = X_train.shape[1]
X_train = X_train.values.reshape((X_train.shape[0], timesteps, 1))
X_test = X_test.values.reshape((X_test.shape[0], timesteps, 1))
y_train = y_train.values.reshape((y_train.shape[0], 4))
y_test = y_test.values.reshape((y_test.shape[0], 4))

In [3]:
from keras.layers import Input, Dense, LSTM, Conv1D, MaxPooling1D, Flatten, GRU
from keras.models import Model
from keras.callbacks import EarlyStopping
import time
timesteps = X_train.shape[1]

# Define the input layer
input_layer = Input(shape=(timesteps, 1))

# Define the CNN layer
cnn_layer = Conv1D(32, kernel_size=3, activation='relu')(input_layer)
cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)

# Define the LSTM layer
lstm_layer = LSTM(50)(cnn_layer)

# Define the output layer
output_layer = Dense(4)(lstm_layer)

# Create the model
model_l = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_l.compile(optimizer='adam', loss='mean_squared_error')
start = time.time()

model_l.load_weights("../CNN_LSTM.h5")

## Frame-wise Prediction

In [1]:
from ultralytics import YOLO

import time
import torch
import cv2
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np

from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker

deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=70)

In [4]:
# Define the video path
video_path = 'vid.mp4'

cap = cv2.VideoCapture(video_path)

# Get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'output.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

device = torch.device('cpu')
frames = []

unique_track_ids = set()

In [ ]:
i = 0
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()
coords_array = [[[0],[1],[0],[0],[0]]]*100
while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        
        og_frame = frame
        frame = og_frame.copy()

        model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

        results = model(frame, device="cpu", classes=3, conf=0.6 )

        class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

        for result in results:
            boxes = result.boxes  # Boxes object for bbox outputs
            probs = result.probs  # Class probabilities for classification outputs
            cls = boxes.cls.tolist()  # Convert tensor to list
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh  # box with xywh format, (N, 4)
            for class_index in cls:
                class_name = class_names[int(class_index)]
                print("Class:", class_name)

        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh
        bboxes_xywh = xywh.cpu().numpy()
        bboxes_xywh = np.array(bboxes_xywh, dtype=float)
        
        tracks = tracker.update(bboxes_xywh, conf, og_frame)
        
        for track in tracker.tracker.tracks:
            track_id = track.track_id
            hits = track.hits
            x1, y1, x2, y2 = track.to_tlbr()  # Get bounding box coordinates in (x1, y1, x2, y2) format
            w = x2 - x1  # Calculate width
            h = y2 - y1  # Calculate height
            
            #print(track_id)
            """if(coords_array[track_id]==None):
                print(track_id,'\n',coords_array[track_id])"""
                
            if(len(coords_array[track_id]) == 37):
                #print("finale",coords_array[track_id])
                temp=[]
                temp.append(coords_array[track_id])
                y_pred = model_l.predict(temp)
                #print("Hello",y_pred)
                pred_x = int((y_pred[0][0]+y_pred[0][1])/2)
                pred_y = int((y_pred[0][2]+y_pred[0][3])/2)
                
                prev_x = int((coords_array[track_id][35][0]+coords_array[track_id][33][0])/2)
                prev_y = int((coords_array[track_id][36][0]+coords_array[track_id][34][0])/2)
                cv2.line(og_frame,[prev_x,prev_y],[pred_x,pred_y],color = (0,255,0),thickness=2)
                coords_array[track_id] = coords_array[track_id][:5]
                #print(coords_array[track_id])
            else:               
                np_array = xyxy
                reshaped_array = np.reshape(np_array, (-1, 1))
                #print("Reshaped_array",reshaped_array)
                #print("Coords_array",coords_array[track_id])
                for x in reshaped_array:
                    coords_array[track_id].append([int(x)])  
            
            # Set color values for red, blue, and green
            red_color = (0, 0, 255)  # (B, G, R)
            blue_color = (255, 0, 0)  # (B, G, R)
            green_color = (0, 255, 0)  # (B, G, R)

            # Determine color based on track_id
            color_id = track_id % 3
            if color_id == 0:
                color = red_color
            elif color_id == 1:
                color = blue_color
            else:
                color = green_color
            
            
            cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 1)

            text_color = (0, 0, 0)  # Black color for text
            cv2.putText(og_frame, f"{class_name}-{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

            # Add the track_id to the set of unique track IDs
            unique_track_ids.add(track_id)
            
            

        # Update the person count based on the number of unique track IDs
        person_count = len(unique_track_ids)

        # Update FPS and place on frame
        current_time = time.perf_counter()
        elapsed = (current_time - start_time)
        counter += 1
        if elapsed > 1:
            fps = counter / elapsed
            counter = 0
            start_time = current_time

        # Draw person count on frame
        cv2.putText(og_frame, f"Bike Count: {person_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

        # Append the frame to the list
        frames.append(og_frame)

        # Write the frame to the output video file
        out.write(og_frame)

        # Show the frame
        cv2.imshow("Video",og_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
             break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 2 motorcycles, 68.9ms
Speed: 3.8ms preprocess, 68.9ms inference, 6.2ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 2 motorcycles, 65.0ms
Speed: 1.7ms preprocess, 65.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 1 motorcycle, 59.4ms
Speed: 1.5ms preprocess, 59.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
1/1 [==============================] - 0s 314ms/step

0: 384x640 (no detections), 57.3ms
Speed: 1.5ms preprocess, 57.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 61.5ms
Speed: 1.6ms preprocess, 61.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 92.8ms
Speed: 1.6ms preprocess, 92.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 60.8ms
Speed: 1.5ms preprocess, 60.8m

Class: motorcycle

0: 384x640 3 motorcycles, 93.3ms
Speed: 1.9ms preprocess, 93.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle
Class: motorcycle
1/1 [==============================] - 0s 29ms/step

0: 384x640 2 motorcycles, 62.1ms
Speed: 1.6ms preprocess, 62.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 2 motorcycles, 63.5ms
Speed: 1.7ms preprocess, 63.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 1 motorcycle, 60.7ms
Speed: 1.6ms preprocess, 60.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle

0: 384x640 1 motorcycle, 63.5ms
Speed: 1.4ms preprocess, 63.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle

0: 384x640 3 motorcycles, 57.7ms
Speed: 1.6ms preprocess, 57.7ms inference, 0.5ms postprocess per image at shape (

Class: motorcycle
Class: motorcycle
1/1 [==============================] - 0s 23ms/step

0: 384x640 2 motorcycles, 56.8ms
Speed: 1.7ms preprocess, 56.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle
1/1 [==============================] - 0s 21ms/step

0: 384x640 2 motorcycles, 62.9ms
Speed: 1.6ms preprocess, 62.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 2 motorcycles, 69.1ms
Speed: 1.6ms preprocess, 69.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 3 motorcycles, 54.7ms
Speed: 1.6ms preprocess, 54.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle
Class: motorcycle

0: 384x640 3 motorcycles, 56.3ms
Speed: 1.6ms preprocess, 56.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle
Class: 

Speed: 1.7ms preprocess, 59.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 2 motorcycles, 61.4ms
Speed: 1.7ms preprocess, 61.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 3 motorcycles, 56.2ms
Speed: 1.6ms preprocess, 56.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle
Class: motorcycle
1/1 [==============================] - 0s 22ms/step

0: 384x640 3 motorcycles, 67.3ms
Speed: 1.5ms preprocess, 67.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle
Class: motorcycle

0: 384x640 3 motorcycles, 63.1ms
Speed: 1.6ms preprocess, 63.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle
Class: motorcycle

0: 384x640 3 motorcycles, 67.9ms
Speed: 1.8ms preprocess, 67.9ms inference, 0.6ms postproces


0: 384x640 2 motorcycles, 61.2ms
Speed: 1.3ms preprocess, 61.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 2 motorcycles, 58.8ms
Speed: 1.6ms preprocess, 58.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 2 motorcycles, 58.8ms
Speed: 1.5ms preprocess, 58.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 1 motorcycle, 69.2ms
Speed: 2.3ms preprocess, 69.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
1/1 [==============================] - 0s 29ms/step

0: 384x640 2 motorcycles, 63.0ms
Speed: 2.1ms preprocess, 63.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle
1/1 [==============================] - 0s 23ms/step

0: 384x640 2 motorcycles, 72.1ms
Speed: 1.5ms preprocess, 72.1ms inference, 0.6

Speed: 2.0ms preprocess, 67.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 2 motorcycles, 57.6ms
Speed: 1.5ms preprocess, 57.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 2 motorcycles, 69.3ms
Speed: 1.6ms preprocess, 69.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 4 motorcycles, 56.9ms
Speed: 1.5ms preprocess, 56.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle
Class: motorcycle
Class: motorcycle
1/1 [==============================] - 0s 32ms/step

0: 384x640 2 motorcycles, 54.9ms
Speed: 2.0ms preprocess, 54.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 2 motorcycles, 55.9ms
Speed: 1.6ms preprocess, 55.9ms inference, 0.9ms postprocess per image at sha

Class: motorcycle

0: 384x640 1 motorcycle, 65.8ms
Speed: 1.5ms preprocess, 65.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle

0: 384x640 1 motorcycle, 64.4ms
Speed: 1.5ms preprocess, 64.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle

0: 384x640 1 motorcycle, 67.4ms
Speed: 1.7ms preprocess, 67.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle

0: 384x640 1 motorcycle, 61.7ms
Speed: 1.6ms preprocess, 61.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle

0: 384x640 2 motorcycles, 62.8ms
Speed: 1.8ms preprocess, 62.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 1 motorcycle, 55.8ms
Speed: 1.6ms preprocess, 55.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle

0: 384x640 1 motorcycle, 66.5ms
Speed: 1.7ms preprocess, 66.5ms inference,

Class: motorcycle
Class: motorcycle

0: 384x640 3 motorcycles, 75.6ms
Speed: 2.0ms preprocess, 75.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle
Class: motorcycle
1/1 [==============================] - 0s 79ms/step

0: 384x640 3 motorcycles, 66.7ms
Speed: 1.5ms preprocess, 66.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle
Class: motorcycle
1/1 [==============================] - 0s 30ms/step

0: 384x640 2 motorcycles, 59.4ms
Speed: 1.6ms preprocess, 59.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 2 motorcycles, 66.9ms
Speed: 1.6ms preprocess, 66.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle
1/1 [==============================] - 0s 20ms/step

0: 384x640 3 motorcycles, 65.0ms
Speed: 1.8ms preprocess, 65.0ms inference, 1.0ms postprocess per image

Class: motorcycle

0: 384x640 1 motorcycle, 65.4ms
Speed: 1.5ms preprocess, 65.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle

0: 384x640 1 motorcycle, 67.1ms
Speed: 3.3ms preprocess, 67.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle

0: 384x640 2 motorcycles, 64.0ms
Speed: 1.6ms preprocess, 64.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 2 motorcycles, 60.4ms
Speed: 1.6ms preprocess, 60.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle
1/1 [==============================] - 0s 28ms/step

0: 384x640 2 motorcycles, 65.4ms
Speed: 1.6ms preprocess, 65.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Class: motorcycle
Class: motorcycle

0: 384x640 2 motorcycles, 79.4ms
Speed: 1.9ms preprocess, 79.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Cl